In [0]:
# Import Data
aisles = spark.read.csv("/FileStore/tables/aisles.csv", header=True, inferSchema=True)
departments = spark.read.csv("/FileStore/tables/departments.csv", header=True, inferSchema=True)
order_products_prior = spark.read.csv("/FileStore/tables/order_products__prior.csv", header=True, inferSchema=True)
order_products_train = spark.read.csv("/FileStore/tables/order_products__train.csv", header=True, inferSchema=True)
orders = spark.read.csv("/FileStore/tables/orders.csv", header=True, inferSchema=True)
products = spark.read.csv("/FileStore/tables/products.csv", header=True, inferSchema=True)

# Create Temporary Tables
aisles.createOrReplaceTempView("aisles")
departments.createOrReplaceTempView("departments")
order_products_prior.createOrReplaceTempView("order_products_prior")
order_products_train.createOrReplaceTempView("order_products_train")
orders.createOrReplaceTempView("orders")
products.createOrReplaceTempView("products")

In [0]:
# Organize the data by shopping basket
from pyspark.sql.functions import collect_set, col, count
rawData = spark.sql("select p.product_name, o.order_id from products p inner join order_products_train o where o.product_id = p.product_id")
baskets = rawData.groupBy('order_id').agg(collect_set('product_name').alias('items'))
baskets.createOrReplaceTempView('baskets')

In [0]:
display(baskets)

order_id,items
762,"List(Organic Cucumber, Organic Romaine Lettuce, Celery Hearts, Organic Strawberries)"
844,"List(Organic Red Radish, Bunch, Baby Spinach, Organic Shredded Carrots, Granny Smith Apples, Green Beans, Cheese Pizza Snacks, Garlic Couscous)"
988,"List(Whipped Light Cream, Original, Complete ActionPacs Lemon Burst Dishwasher Detergent, Classic Vanilla Coffee Creamer, Natural Vanilla Ice Cream)"
1139,"List(Cinnamon Rolls with Icing, Red Vine Tomato, Picnic Potato Salad, Flaky Biscuits, Organic Strawberries, Organic Bakery Hamburger Buns Wheat - 8 CT, Buttermilk Biscuits, Banana, Guacamole)"
1143,"List(Water, Natural Premium Coconut Water, Organic Red Radish, Bunch, Organic Capellini Whole Wheat Pasta, Organic Raspberries, Calming Lavender Body Wash, Organic Garlic, Rustic Baguette, Organic Brussel Sprouts, Organic Butterhead (Boston, Butter, Bibb) Lettuce, Organic Blueberries, Spring Water, Large Lemon, Basil Pesto, Baby Arugula, Organic Hass Avocado, Unscented Long Lasting Stick Deodorant)"
1280,"List(Vanilla Soy Milk, French Vanilla Creamer, Organic Half & Half, Lactose Free Half & Half, Organic Whole Milk)"
1342,"List(Raw Shrimp, Seedless Cucumbers, Versatile Stain Remover, Organic Strawberries, Organic Mandarins, Chicken Apple Sausage, Pink Lady Apples, Bag of Organic Bananas)"
1350,"List(Mocha Frappucino Chilled Coffee Drink, Bare Fruit Banana Chips, Pressed Cool Pineapple, Ground Cinnamon, Lemon Love Juice Drink, Chia Sweet Peach Smoothie, Green Apple Chips, Sea Salt Chickpeas, Plus Lotion Facial Tissues, Organic Insect Repellent Fresh Natural Scent, Strawberry Banana Juice)"
1468,"List(Pomegranate Seeds, Organic Red Radish, Bunch, Natural Mini Pork Pepperoni, Cage Free Grade AA Large White Eggs, Bartlett Pears, Organic Red Potato, Organic Ginger Root, Banana, Red Peppers, Active Dry Yeast, Organic Lacinato (Dinosaur) Kale, Organic Baby Broccoli, Carrots, Fresh Cauliflower, Organic English Cucumber, Organic Grape Tomatoes, Organic Hass Avocado)"
1591,"List(Cracked Wheat, Strawberry Rhubarb Yoghurt, Organic Bunny Fruit Snacks Berry Patch, Goodness Grapeness Organic Juice Drink, Honey Graham Snacks, Spinach, Granny Smith Apples, Oven Roasted Turkey Breast, Pure Vanilla Extract, Chewy 25% Low Sugar Chocolate Chip Granola, Banana, Original Turkey Burgers Smoke Flavor Added, Twisted Tropical Tango Organic Juice Drink, Navel Oranges, Lower Sugar Instant Oatmeal Variety, Ultra Thin Sliced Provolone Cheese, Natural Vanilla Ice Cream, Cinnamon Multigrain Cereal, Garlic, Goldfish Pretzel Baked Snack Crackers, Original Whole Grain Chips, Medium Scarlet Raspberries, Lemon Yogurt, Original Patties (100965) 12 Oz Breakfast, Nutty Bars, Strawberry Banana Smoothie, Green Machine Juice Smoothie, Coconut Dreams Cookies, Buttermilk Waffles, Uncured Genoa Salami, Organic Greek Whole Milk Blended Vanilla Bean Yogurt)"


In [0]:
%scala
import org.apache.spark.ml.fpm.FPGrowth

// Extract out the items 
val baskets_ds = spark.sql("select items from baskets").as[Array[String]].toDF("items")

// Use FPGrowth
val fpgrowth = new FPGrowth().setItemsCol("items").setMinSupport(0.001).setMinConfidence(0)
val model = fpgrowth.fit(baskets_ds)

import org.apache.spark.ml.fpm.FPGrowth
baskets_ds: org.apache.spark.sql.DataFrame = [items: array<string>]
fpgrowth: org.apache.spark.ml.fpm.FPGrowth = fpgrowth_058b87c34e5c
model: org.apache.spark.ml.fpm.FPGrowthModel = FPGrowthModel: uid=fpgrowth_058b87c34e5c, numTrainingRecords=131209

In [0]:
%scala
// Display frequent itemsets
val mostPopularItemInABasket = model.freqItemsets
mostPopularItemInABasket.createOrReplaceTempView("mostPopularItemInABasket")

mostPopularItemInABasket: org.apache.spark.sql.DataFrame = [items: array<string>, freq: bigint]

In [0]:
%sql
select items, freq from mostPopularItemInABasket where size(items) > 2 order by freq desc limit 20

items,freq
"List(Organic Hass Avocado, Organic Strawberries, Bag of Organic Bananas)",710
"List(Organic Raspberries, Organic Strawberries, Bag of Organic Bananas)",649
"List(Organic Baby Spinach, Organic Strawberries, Bag of Organic Bananas)",587
"List(Organic Raspberries, Organic Hass Avocado, Bag of Organic Bananas)",531
"List(Organic Hass Avocado, Organic Baby Spinach, Bag of Organic Bananas)",497
"List(Organic Avocado, Organic Baby Spinach, Banana)",484
"List(Organic Avocado, Large Lemon, Banana)",477
"List(Limes, Large Lemon, Banana)",452
"List(Organic Cucumber, Organic Strawberries, Bag of Organic Bananas)",424
"List(Limes, Organic Avocado, Large Lemon)",389


In [0]:
%scala
// Display generated association rules.
val ifThen = model.associationRules
ifThen.createOrReplaceTempView("ifThen")

ifThen: org.apache.spark.sql.DataFrame = [antecedent: array<string>, consequent: array<string> ... 3 more fields]

In [0]:
%sql
select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence from ifThen order by confidence desc limit 20

antecedent (if),consequent (then),confidence
"List(Organic Raspberries, Organic Hass Avocado, Organic Strawberries)",List(Bag of Organic Bananas),0.5984251968503937
"List(Organic Cucumber, Organic Hass Avocado, Organic Strawberries)",List(Bag of Organic Bananas),0.546875
"List(Organic Kiwi, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5459770114942529
"List(Organic Navel Orange, Organic Raspberries)",List(Bag of Organic Bananas),0.5412186379928315
"List(Yellow Onions, Strawberries)",List(Banana),0.5357142857142857
"List(Organic Whole String Cheese, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5314685314685315
"List(Organic Navel Orange, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5283018867924528
"List(Organic Raspberries, Organic Hass Avocado)",List(Bag of Organic Bananas),0.521099116781158
"List(Organic D'Anjou Pears, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5170454545454546
"List(Organic Unsweetened Almond Milk, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5141065830721003
